In [1]:
import torch
import os
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data

Notice we call 'test' the validation set

In [2]:
from pathlib import Path

# get data PATH from current directory
PATH = os.path.join(Path(os.getcwd()).parent, 'data')


PATH_DATASET = os.path.join(PATH, 'datasets')

PATH_VOCAB = os.path.join(PATH, 'vocabularies')

PATH_MODEL = os.path.join(PATH, 'models')

PATH_STATS = os.path.join(PATH, 'stats')


In [ ]:
# Load training and test set in pandas dataframe

train = pd.read_hdf(os.path.join(PATH_DATASET,'train_set.h5'))
test = pd.read_hdf(os.path.join(PATH_DATASET,'test_set.h5'))

train_rel = pd.read_hdf(os.path.join(PATH_DATASET,'train_set_rel.h5'))
train_unrel = pd.read_hdf(os.path.join(PATH_DATASET,'train_set_unrel.h5'))

# Load vocabulary

file = open(os.path.join(PATH_VOCAB,'vocab_0.0015.pkl'), 'rb')
vocab = pickle.load(file)


# Load classes dictionary

file = open(os.path.join(PATH,'classes_dict.pkl'), 'rb')
classes_dict = pickle.load(file)

cat_dict = classes_dict['class_stoi']
cat_dict_inv = classes_dict['class_itos']
dict_count = classes_dict['class_count']

# Vectorize

Make BoW using my tokenizer.

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# Define the vectorizer to generate the BoW

vectorizer_tok = CountVectorizer(
                             analyzer = 'word',
                             strip_accents='unicode',
                             vocabulary=vocab
                            )


# Reset indexes of the training set before vectorizer
train = train.reset_index()

# Vectorize the training and test_set
vectors_train = vectorizer_tok.fit_transform(train.title)

vectors_test = vectorizer_tok.transform(test.title)


y_train = train.category.values

X_test = vectors_test
y_test = test.category.values

# Util functions


## Sampling

In [ ]:
# Split training set by categories
list_train_rel = list(train_rel.groupby('category'))
list_train_unrel = list(train_unrel.groupby('category'))

class_rel = train_rel.groupby('category').count().title.index
class_unrel = train_unrel.groupby('category').count().title.index


file = open(os.path.join(PATH,'classes_weights.pkl'), 'rb')
weights = pickle.load(file)


def numero(weight, num_examples, proporcion):
    
    if weight==0:
        return 0
        
    return int(min(max(1, round(num_examples*weight*proporcion)), num_examples-1))


def index_sample(list_train_rel, list_train_unrel, num_samples, proporcion):
    list_idx = []
    
    # Unreliable indexes
    
    for i in range(len(class_unrel)):
        # Convention index
        idx = cat_dict[class_unrel[i]]
        number = num_samples - numero(weights[idx,2], num_samples, proporcion)
        list_idx += list(np.random.choice(list_train_unrel[i][1].index, size=number))
    
    # Reliable indexes
    for i,category in enumerate(class_rel):
        
        idx = cat_dict[class_rel[i]]
        number = numero(weights[idx,2], num_samples, proporcion)
        list_idx += list(np.random.choice(list_train_rel[i][1].index, size=number))
    
    
    return list_idx

## Predictions in CPU

Functions to calculate bacc and accuracy using CPU.

We train the model using GPU, but we don't have enough memory to run the predictions in the validation set while training. So that, at the end of each epoch we set the model to the cpu, make predictions in the validation set and then set the model to gpu and continue training

In [ ]:
import torch
import numpy as np
from scipy.sparse import coo_matrix
from numpy import array
from sklearn.metrics import balanced_accuracy_score, accuracy_score


def report(model, X_test, y_test):
    
    x_tensor = csr_to_tensor(X_test)
    model_cpu = model.to('cpu')
    outputs = model_cpu(x_tensor)
    _, predicted = torch.max(outputs.data, 1)
    predicted.cpu().numpy()
    y_label = []
    for val in y_test:
        y_label.append(cat_dict[val])
    y_label = np.array(y_label)
    
    return balanced_accuracy_score(y_label, predicted), accuracy_score(y_label, predicted)


def csr_to_tensor(X_sample):
    coo = coo_matrix(X_sample)

    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

def numerate(y):
    index = []
    for i in y:
        index.append(cat_dict[i])
    return np.array(index)


# Training

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pickle
from sklearn.utils import shuffle




input_size = vectors_train.shape[1]
num_classes = len(train.category.unique())
H = num_classes

H1 = 2*num_classes

vec_prop = [10]
vec_num_epochs = [35] 
vec_learning_rate = [0.0005]


for learning_rate, num_epochs, prop in zip(vec_learning_rate, vec_num_epochs, vec_prop):

    ## Define model
    model = torch.nn.Sequential(
          torch.nn.Linear(input_size, H1),
          torch.nn.ReLU(),
          torch.nn.Dropout(p=0.75),
          torch.nn.Linear(H1, num_classes),
    ).to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Some training stats
    model_stats = {}
    model_stats['epoch'] = []
    model_stats['avg_loss'] = []
    model_stats['accuracy'] = []
    model_stats['bacc'] = []

    # Record maximum BACC value
    max_bacc = 0
    num_max = 0
    
    
    # Epochs aren't 'real epochs' in which we use all the training set. 
    # Instead each epochs is a collection of 1000 mini-batches generated using the sampling defined above
    # Each epoch only defines when and how many times we evaluate the model using the validation set.
    for epoch in range(num_epochs):
        
        model.train()
        
        list_loss = []
        
        for i in range(1000):
            
            # List of index of the minibatch using the function defined above
            list_idx = index_sample(list_train_rel, list_train_unrel, 10, prop)
            
            X = vectors_train[list_idx, :]
            y = y_train[list_idx]
            
            X_sample, y_sample = shuffle(X, y)

            x_tensor = csr_to_tensor(X_sample).to(device)

            labels = torch.from_numpy(numerate(y_sample)).long().to(device)

            outputs = model(x_tensor)
            #images.to('cpu')
            #torch.cuda.empty_cache()
            
            
            loss = criterion(outputs, labels)
            lo = loss.item()
            list_loss.append(lo)
            # Test
            #outputs.to('cpu')
            #torch.cuda.empty_cache()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, lo )) ##ss.item()))

        ## Statistics

        # Bacc and accuracy
        
        bacc, accuracy = report(model.eval(), X_test, y_test)
        print(bacc,accuracy)
        model.to(device)

        # Save in the model_stats dictionary
        model_stats['epoch'].append(epoch)
        model_stats['bacc'].append(bacc)
        model_stats['accuracy'].append(accuracy)
        model_stats['avg_loss'].append(np.array(list_loss).mean())
        

        if bacc > max_bacc:
            max_bacc = bacc
            torch.save(model, os.path.join(PATH_MODEL,'model_00015_prop{}_lr{}.pt'.format(learning_rate)))


        with open( os.path.join(PATH_STATS,'stats_00015_prop{}_lr{}.pt'.format(learning_rate)), 'wb') as f:
            pickle.dump(model_stats, f)     